Kernel : Python 3.10.12

source myenv/bin/activate

# 0. Define paths

**data container mount**

In [0]:
storage_account_name = "projet8images"
container_name = "imagesdata"
storage_account_key = "zez1wQv/LGqyCCNn1LAKv0jTyT29ilq0/ccqsEHT1X4CyKOlHa7bOAPu7xm5xglPmnCU1UVFvRnb+AStFfOftw=="

dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point = "/mnt/projet8",
    extra_configs = {"fs.azure.account.key.projet8images.blob.core.windows.net": storage_account_key}
)


**results container mount**

In [0]:
storage_account_name = "projet8images"
container_name = "results"
storage_account_key = "zez1wQv/LGqyCCNn1LAKv0jTyT29ilq0/ccqsEHT1X4CyKOlHa7bOAPu7xm5xglPmnCU1UVFvRnb+AStFfOftw=="

dbutils.fs.mount(
    source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
    mount_point = "/mnt/results",
    extra_configs = {"fs.azure.account.key.projet8images.blob.core.windows.net": storage_account_key}
)

**show the path_data for the images**

In [0]:
dbutils.fs.ls("/mnt/projet8/fruits/fruits/fruits-360-original-size/fruits-360-original-size/Test/")

In [0]:
dbutils.fs.ls("/mnt/results/")

In [0]:
os.getcwd()

# I. Librairies & packages

In [0]:
# File system manangement
#import os
#from os import listdir
#import zipfile
#from zipfile import ZipFile

#import pyspark.pandas as ps
import pyspark.pandas as ps
from PIL import Image
import numpy as np
import io

import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql import SparkSession

#second PCA
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.sql.functions import col, size, udf, lit
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import VectorUDT, Vectors
import matplotlib.pyplot as plt

## Installations

explorer.exe .

In [0]:
#!pip install Pandas pillow tensorflow pyspark pyarrow

In [0]:
#pip install matplotlib

# II. Extract zip files + define paths

In [0]:
#PATH = os.getcwd()

In [0]:
#PATH

In [0]:
PATH_Data = "/mnt/projet8/fruits/fruits/fruits-360-original-size/fruits-360-original-size/Test/"

In [0]:
PATH_Result = '/mnt/results/'

## Open zip files

with zipfile.ZipFile("Mode_opératoire.zip","r") as zip_ref:
    zip_ref.extractall("targetdir")

In [0]:
#"C:\Users\denis\Downloads\fruits.zip"

with zipfile.ZipFile("fruits.zip","r") as zip_ref:
    zip_ref.extractall("images")

# III. Notebook from other

## III.1 Création de la SparkSession

L’application Spark est contrôlée grâce à un processus de pilotage (driver process) appelé **SparkSession**. <br />
<u>Une instance de **SparkSession** est la façon dont Spark exécute les fonctions définies par l’utilisateur <br />
dans l’ensemble du cluster</u>. <u>Une SparkSession correspond toujours à une application Spark</u>.

<u>Ici nous créons une session spark en spécifiant dans l'ordre</u> :
 1. un **nom pour l'application**, qui sera affichée dans l'interface utilisateur Web Spark "**P8**"
 2. que l'application doit s'exécuter **localement**. <br />
   Nous ne définissons pas le nombre de cœurs à utiliser (comme .master('local[4]) pour 4 cœurs à utiliser), <br />
   nous utiliserons donc tous les cœurs disponibles dans notre processeur.<br />
 3. une option de configuration supplémentaire permettant d'utiliser le **format "parquet"** <br />
   que nous utiliserons pour enregistrer et charger le résultat de notre travail.
 4. vouloir **obtenir une session spark** existante ou si aucune n'existe, en créer une nouvelle

In [0]:
spark = (SparkSession
             .builder
             .appName('P8')
             .master('local')
             .config("spark.sql.parquet.writeLegacyFormat", 'true')
             .getOrCreate()
)

<u>Nous créons également la variable "**sc**" qui est un **SparkContext** issue de la variable **spark**</u> :

In [0]:
sc = spark.sparkContext

<u>Affichage des informations de Spark en cours d'execution</u> :

In [0]:
spark

## III.2 Traitement des données

<u>Dans la suite de notre flux de travail, <br />
nous allons successivement</u> :
1. Préparer nos données
    1. Importer les images dans un dataframe **pandas UDF**
    2. Associer aux images leur **label**
    3. Préprocesser en **redimensionnant nos images pour <br />
       qu'elles soient compatibles avec notre modèle**
2. Préparer notre modèle
    1. Importer le modèle **MobileNetV2**
    2. Créer un **nouveau modèle** dépourvu de la dernière couche de MobileNetV2
3. Définir le processus de chargement des images et l'application <br />
   de leur featurisation à travers l'utilisation de pandas UDF
3. Exécuter les actions d'extraction de features
4. Enregistrer le résultat de nos actions
5. Tester le bon fonctionnement en chargeant les données enregistrées




### III.2.1 Chargement des données

Les images sont chargées au format binaire, ce qui offre, <br />
plus de souplesse dans la façon de prétraiter les images.

Avant de charger les images, nous spécifions que nous voulons charger <br />
uniquement les fichiers dont l'extension est **jpg**.

Nous indiquons également de charger tous les objets possibles contenus <br />
dans les sous-dossiers du dossier communiqué.

In [0]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

<u>Affichage des 5 premières images contenant</u> :
 - le path de l'image
 - la date et heure de sa dernière modification
 - sa longueur
 - son contenu encodé en valeur hexadécimal

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [0]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

### III.7.2 Préparation du modèle

Je vais utiliser la technique du **transfert learning** pour extraire les features des images.<br />
J'ai choisi d'utiliser le modèle **MobileNetV2** pour sa rapidité d'exécution comparée <br />
à d'autres modèles comme *VGG16* par exemple.

Pour en savoir plus sur la conception et le fonctionnement de MobileNetV2, <br />
je vous invite à lire [cet article](https://towardsdatascience.com/review-mobilenetv2-light-weight-model-image-classification-8febb490e61c).

<u>Voici le schéma de son architecture globale</u> : 

![Architecture de MobileNetV2](img/mobilenetv2_architecture.png)

Il existe une dernière couche qui sert à classer les images <br />
selon 1000 catégories que nous ne voulons pas utiliser.<br />
L'idée dans ce projet est de récupérer le **vecteur de caractéristiques <br />
de dimensions (1,1,1280)** qui servira, plus tard, au travers d'un moteur <br />
de classification à reconnaitre les différents fruits du jeu de données.

Comme d'autres modèles similaires, **MobileNetV2**, lorsqu'on l'utilise <br />
en incluant toutes ses couches, attend obligatoirement des images <br />
de dimension (224,224,3). Nos images étant toutes de dimension (100,100,3), <br />
nous devrons simplement les **redimensionner** avant de les confier au modèle.

<u>Dans l'odre</u> :
 1. Nous chargeons le modèle **MobileNetV2** avec les poids **précalculés** <br />
    issus d'**imagenet** et en spécifiant le format de nos images en entrée
 2. Nous créons un nouveau modèle avec:
  - <u>en entrée</u> : l'entrée du modèle MobileNetV2
  - <u>en sortie</u> : l'avant dernière couche du modèle MobileNetV2

In [0]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

### /!\ added step!!

In [0]:
#bc_model_weights = sc.broadcast(model.get_weights())

In [0]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

Affichage du résumé de notre nouveau modèle où nous constatons <br />
que <u>nous récupérons bien en sortie un vecteur de dimension (1, 1, 1280)</u> :

In [0]:
new_model.summary()

In [0]:
brodcast_weights = sc.broadcast(new_model.get_weights())

<u>Mettons cela sous forme de fonction</u> :

In [0]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

### III.7.3 Définition du processus de chargement des images et application <br/>de leur featurisation à travers l'utilisation de pandas UDF

Ce notebook définit la logique par étapes, jusqu'à Pandas UDF.

<u>L'empilement des appels est la suivante</u> :

- Pandas UDF
  - featuriser une série d'images pd.Series
   - prétraiter une image

In [0]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

### III.7.4 Exécution des actions d'extraction de features

Les Pandas UDF, sur de grands enregistrements (par exemple, de très grandes images), <br />
peuvent rencontrer des erreurs de type Out Of Memory (OOM).<br />
Si vous rencontrez de telles erreurs dans la cellule ci-dessous, <br />
essayez de réduire la taille du lot Arrow via 'maxRecordsPerBatch'

Je n'utiliserai pas cette commande dans ce projet <br />
et je laisse donc la commande en commentaire.

In [0]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

Nous pouvons maintenant exécuter la featurisation sur l'ensemble de notre DataFrame Spark.<br />
<u>REMARQUE</u> : Cela peut prendre beaucoup de temps, tout dépend du volume de données à traiter. <br />

Notre jeu de données de **Test** contient **22819 images**. <br />
Cependant, dans l'exécution en mode **local**, <br />
nous <u>traiterons un ensemble réduit de **330 images**</u>.

In [0]:
features_df = images.repartition(20).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

<u>Rappel du PATH où seront inscrits les fichiers au format "**parquet**" <br />
contenant nos résultats, à savoir, un DataFrame contenant 3 colonnes</u> :
 1. Path des images
 2. Label de l'image
 3. Vecteur de caractéristiques de l'image

In [0]:
print(PATH_Result)

In [0]:
features_df.write.mode("overwrite").parquet(PATH_Result)

## III.8 Chargement des données enregistrées et validation du résultat

<u>On charge les données fraichement enregistrées dans un **DataFrame Pandas**</u> :

In [0]:
df = ps.read_parquet(PATH_Result, engine='pyarrow')

<u>On affiche les 5 premières lignes du DataFrame</u> :

In [0]:
df.head()

<u>On valide que la dimension du vecteur de caractéristiques des images est bien de dimension 1280</u> :

In [0]:
df.loc[0,'features'].shape

In [0]:
df.shape

Nous venons de valider le processus sur un jeu de données allégé en local <br />
où nous avons simulé un cluster de machines en répartissant la charge de travail <br />
sur différents cœurs de processeur au sein d'une même machine.

Nous allons maintenant généraliser le processus en déployant notre solution <br />
sur un réel cluster de machines et nous travaillerons désormais sur la totalité <br />
des 22819 images de notre dossier "Test".

# III.9 Dimension reduction (PCA)

In [0]:
type(df)

In [0]:
df.loc[0, 'features']

In [0]:
df.features.to_numpy()

## First try : discarded

In [0]:
data = pd.DataFrame(data=list(df.features))

In [0]:
data

In [0]:
col = ['col'+str(ele) for ele in data.columns]

In [0]:
data.columns = col

In [0]:
data.to_csv(PATH_Result+'/data.csv', index=False)

In [0]:
from pyspark.ml.feature import RFormula, PCA
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.ml.linalg import Vectors
#from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [0]:
data = sqlContext.read.load(PATH_Result+'/data.csv', format='com.databricks.spark.csv', delimiter = ',', header='true', inferSchema='true')

In [0]:
type(data)

In [0]:
data

In [0]:
data.show()

In [0]:
#dataPCA = PCA(k=20, inputCol=str(data.columns), outputCol="pcaFeatures")
#model = dataPCA.fit(data)

In [0]:
data = RFormula(formula=" ~ {0}".format(" + ".join(data.columns))).fit(data).transform(data)

In [0]:
data

In [0]:
dataPCA = PCA(k=50, inputCol=str(data.columns), outputCol="pcaFeatures")
#model = dataPCA.fit(data)

In [0]:
dataPCA

In [0]:
dataPCA.setInputCol("features").fit(data).transform(data)

In [0]:
pcaModel = dataPCA.fit(data)

In [0]:
pcaModel.getK

In [0]:
pcaModel.explainedVariance

In [0]:
cumValues = pcaModel.explainedVariance.cumsum() # get the cumulative values

In [0]:
cumValues

In [0]:
# plot the graph 
plt.figure(figsize=(10,8))
plt.plot(range(1,51), cumValues, marker = 'o', linestyle='--')
plt.title('variance by components')
plt.xlabel('num of components')
plt.ylabel('cumulative explained variance')

In [0]:
pcaModel.setOutputCol("output")

In [0]:
data.show()

In [0]:
type(data)

In [0]:
pcaModel.transform(data).collect()[0].output

In [0]:
pcaModel.transform(data).collect()[0]

In [0]:
df_ = pcaModel.transform(data)

In [0]:
type(df_)

In [0]:
df_.show()

In [0]:
df_.select('output').collect()

In [0]:
df2_ = spark.createDataFrame(df_.select('output').collect(),["features"])

In [0]:
df2_.show()

In [0]:
type(df2_)

from pyspark.ml.functions import vector_to_array
df3_ = df2_.withColumn('features', vector_to_array('features'))

In [0]:
#data.count()

In [0]:
# Save the DataFrame to a Parquet file
parquet_path = PATH_Result+'/myparquet'
df2_.write.parquet(parquet_path)

In [0]:
# Read the Parquet file to verify
df_read = spark.read.parquet(parquet_path)
df_read.show()

In [0]:
type(df_read)

In [0]:
df_read.collect()[0]

In [0]:
df_read.collect()[0][0][-1]

## /!\ Second PCA

In [0]:
num_elements = len(features_df.select('features').first()['features'])

In [0]:
num_elements

In [0]:
# fct utilisateur pour convertir ArrayType en VectorUDT
def array_to_vector_udf(array_col):
    return Vectors.dense(array_col)

In [0]:
#enregistrement de la fct utilisateur en tant qu'UDF
array_to_vector = udf(array_to_vector_udf, VectorUDT())
features_df = features_df.withColumn('vector_features', array_to_vector('features'))

In [0]:
pca = PCA(k=num_elements, inputCol='vector_features')
pca_model = pca.fit(features_df)

In [0]:
print(pca_model)

In [0]:
pca_model.setOutputCol('pca_features')

In [0]:
pca_exp = pca_model.explainedVariance

In [0]:
cumValues = pca_exp.cumsum() # get the cumulative values

In [0]:
cumValues[199] #selection of the 200 st components (new_features)

In [0]:
# plot the graph 
plt.figure(figsize=(10,8))
plt.plot(range(1,num_elements+1), cumValues, marker = 'o', linestyle='--')
plt.title('variance by components')
plt.xlabel('num of components')
plt.ylabel('cumulative explained variance')

In [0]:
pca_features = pca_model.transform(features_df)

In [0]:
#pca_features.show()

In [0]:
pca_features.select('pca_features').show()

In [0]:
# extract the 200 st components
# define an user fct to extract the 2 fist elts 
def extract_first_n_udf(features, n):
    return Vectors.dense(features[:n])

In [0]:
extract_first_n = udf(extract_first_n_udf, VectorUDT()) #enregistrer la fct utilisateur en tant q'UDF

In [0]:
#specficy the number to extract (ex:)
n_elements = 200

In [0]:
#appliuqer l'UDF pour extraire les n premiers elts
pca_features = pca_features.withColumn('pca200', extract_first_n(col('pca_features'), lit(n_elements)))

In [0]:
pca_features.columns

In [0]:
features_df = features_df.join(pca_features.select('path', 'pca200'), 'path')

In [0]:
type(features_df)

In [0]:
# Save the DataFrame to a Parquet file
parquet_path = PATH_Result+'/myparquet'
features_df.write.parquet(parquet_path)

In [0]:
# Read the Parquet file to verify
df_read = spark.read.parquet(parquet_path)
df_read.show()

In [0]:
df_ = ps.read_parquet(parquet_path, engine='pyarrow')

In [0]:
df_.head(3)

In [0]:
len(df_.loc[0, 'features']), len(df_.loc[0, 'vector_features']), len(df_.loc[0, 'pca200'])